# 07. 統合演習: RAGとLoRAを組み合わせたGradio UIの作成

これまで学んできたLLM、RAG、LoRAをすべて組み合わせ、一つのアプリケーションを構築します。

## 事前準備
Google Colabで **T4 GPU** になっていることを確認してください。

In [ ]:
# 編集禁止セル
import os
import sys
import torch
import json
from google.colab import drive

# Google Driveのマウント (LoRAアダプタの読み込みに必要)
if not os.path.isdir('/content/drive'): drive.mount('/content/drive')

repo_path = '/content/llm_lab'
if os.path.exists(repo_path): !rm -rf {repo_path}
!git clone -b stable-base https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft trl datasets gradio
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

from src.common import load_llm, generate_text
from src.rag import FaissRAGPipeline
from src.lora import create_lora_model, train_lora
from src.ui import create_gradio_ui
from peft import PeftModel
print('セットアップが完了しました。')

## 1. RAGとモデルの準備

Google Driveに保存されたLoRAアダプタをロードします。もし見つからない場合は、その場で簡易作成を行います。

In [ ]:
# 編集禁止セル
base_model, tokenizer, rag_pipeline, lora_model = None, None, None, None

try:
    # 1. モデルのロード
    base_model, tokenizer = load_llm(use_4bit=True)

    # 2. RAGの準備 (サンプルデータから即時構築)
    rag_pipeline = FaissRAGPipeline()
    rag_pipeline.build_index('data/docs/anime_docs_sample.jsonl')

    # 3. LoRAアダプタのロード (Google Driveから)
    adapter_path = "/content/drive/MyDrive/llm_lab_outputs/demo_adapter/final_adapter"
    
    if not os.path.exists(adapter_path):
        print("Google Driveにアダプタが見つかりません。05を実行していない場合は、その場で作成します（約3分）...")
        tmp_lora = create_lora_model(base_model)
        train_lora(
            model=tmp_lora,
            tokenizer=tokenizer,
            train_dataset_path='data/lora/lora_train_sample.jsonl',
            output_dir="/content/temp_adapter",
            max_steps=30,
            learning_rate=5e-5
        )
        adapter_path = "/content/temp_adapter/final_adapter"
        del tmp_lora
        torch.cuda.empty_cache()
    
    print(f"LoRAアダプタをロードしています: {adapter_path}")
    lora_model = PeftModel.from_pretrained(base_model, adapter_path)
    lora_model.eval()
    print("すべての準備が整いました。")

except Exception as e:
    print(f"エラーが発生しました: {e}")

## 2. UI起動

In [ ]:
# 編集禁止セル
def generate_plain(q): 
    return generate_text(base_model, tokenizer, f"### 指示:\n{q}\n\n### 応答:\n").split("### 応答:")[-1].strip()

def generate_rag(q):
    docs = rag_pipeline.search(q, top_k=2)
    prompt = rag_pipeline.create_prompt_with_context(q, docs)
    res = generate_text(base_model, tokenizer, prompt)
    ans = res.split("回答:")[-1].strip() if "回答:" in res else res.strip()
    return ans, "\n\n".join([d['text'] for d in docs])

def generate_lora(q):
    res = generate_text(lora_model, tokenizer, f"### 指示:\n{q}\n\n### 応答:\n")
    return res.split("### 応答:")[-1].strip()

def generate_rag_lora(q):
    docs = rag_pipeline.search(q, top_k=2)
    prompt = rag_pipeline.create_prompt_with_context(q, docs)
    res = generate_text(lora_model, tokenizer, prompt)
    ans = res.split("回答:")[-1].strip() if "回答:" in res else res.strip()
    return ans, "\n\n".join([d['text'] for d in docs])

sample_queries = [
    ["『星屑のメモリー』の主人公について教えて。", True, False],
    ["『古都の探偵録』のあらすじを教えて。", True, True],
    ["日本の首都は？", False, True],
]

demo = create_gradio_ui(generate_plain, generate_rag, generate_lora, generate_rag_lora, examples=sample_queries)
demo.launch(share=True, debug=True)